In [1]:
library(tidyverse)
bugbank_data_dir = "/well/bag/clme1992/bugbank_data/"
ukb_data_dir = "/well/bag/wilson/ukb/"
gwas_wrkdir = "/well/bag/clme1992/saige_pipe_test"


-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.3     v readr     2.1.4
v forcats   1.0.0     v stringr   1.5.0
v ggplot2   3.4.4     v tibble    3.2.1
v lubridate 1.9.3     v tidyr     1.3.0
v purrr     1.0.2     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# At each step of QC, track the change in the infection cases

resolve.symlink <- function(x) {
    y <- Sys.readlink(x)
    if (any(is.na(y))) {
        stop("Could not resolve symlink ", x)
    }
    ifelse(y == "", x, file.path(dirname(x), y))
}

# configuration file
source("~/.saige_pipe.config")

lg <- list()
lg$bd_RDdata_file <- paste0(config$ukb.derived.dir, "/ukb41482.ukb41376.fields.RData")
lg$bd_not_lost2followup_file <- paste0(config$ukb.derived.dir, "/ukb41482.English-not-lost-to-followup-8-April-2020.txt")
lg$bed_sample_qc_file <- paste0(config$ukbdir, "/v2/qc/ukb_sqc_v2.txt")
lg$withdrawn_eid_file <- paste0(config$bbdatadir, "/w53100_2023-04-25.csv")
# Pre-computed eids for the bed-format genotypes
lg$bed_eid_file <- paste0(config$ukb.derived.dir, "/analysis.bed.eids.txt")
# Individuals with first degree relatives
lg$remrels_file <- paste0(config$ukb.derived.dir, "/ukb41482.English-remove-first-degree-relatives.eids.txt")
# panUKB ancestral files
lg$pan_ukb_file <- paste0(config$panukb.dir, "/Files for retman/all_pops_non_eur_pruned_within_pop_pc_covs.tsv")
lg$pan_ukb_bridge_file <- paste0(config$panukb.dir, "/ukb53100bridge31063.txt")

### load input files ###
system.time(load(lg$bd_RDdata_file))
all_eids <- bd[, "f.eid"]
bd_not_lost2followup <- scan(lg$bd_not_lost2followup_file, what = "logical") == "TRUE"
withdrawn_eid <- scan(lg$withdrawn_eid_file)
# Sample QC
bed_sample_qc <- read.csv(lg$bed_sample_qc_file, sep = " ")
# The corresponding eids
bed_eid <- scan(lg$bed_eid_file)
# Convert to bd_eid order
sample_qc <- bed_sample_qc[match(all_eids, bed_eid), ]
# Close (first degree) relatives
remrels <- scan(lg$remrels_file)
# load and match the panukb data
panukb <- read.csv(lg$pan_ukb_file, sep = "\t")[, c("s", "pop")]
panukb_bridge <- read.csv(lg$pan_ukb_bridge_file, sep = " ", header = F)
bridge_matched <- panukb_bridge[match(panukb$s, panukb_bridge[, 2]), ]
panukb$eid <- bridge_matched[, 1]
panukb_matched <- panukb[match(all_eids, panukb$eid), ]

### assign assessment centre data ###
f.assesscentre <- "f.54.0.0"
assess_centre_England <- c(
    11012, 11021, 11011, 11008, 11024, 11020, 11018, 11010, 11016, 
    11001, 11017, 11009, 11013, 11002, 11007, 11014, 10003, 11006, 
    11025, 11026, 11027, 11028 
)
f_assesscentre <- "f.54.0.0"

   user  system elapsed 
 18.136   2.491  20.707 

In [3]:
# get summary for SGSS
sgss_file = paste0(bugbank_data_dir, "ukb_sgss_extract_refined.csv")
sgss = read.csv(sgss_file, sep = "\t", header = TRUE, stringsAsFactors = FALSE)

# load HES
start_time = Sys.time()
hes_file = paste0(ukb_data_dir, "hes/hesin_diag.latest.txt.gz")
hes = read.table(hes_file, sep = "\t", header = TRUE, stringsAsFactors = FALSE)
end_time = Sys.time()
print(paste0("Time taken to load hes: ", end_time - start_time))

[1] "Time taken to load hes: 24.5665311813354"


In [4]:
# load icd10 data
icd10_desc_file <- paste0(bugbank_data_dir, "pathogen_to_unique_icd10.tsv")
icd10_desc <- read.table(icd10_desc_file, sep = "\t", header = TRUE, stringsAsFactors = FALSE)

# subset hes to only infection related
infect_icd10_codes <- unique(unlist(strsplit(icd10_desc$icd10, split = ",")))
hes_infect <- hes[hes$diag_icd10 %in% infect_icd10_codes, ]

In [36]:
filter_lst <- list()
filter <- rep(T, nrow(bd))
filter_lst[[1]] <- filter
names(filter_lst)[1] <- "all"

filter <- filter & bd_not_lost2followup
filter <- filter & !(all_eids %in% withdrawn_eid)
filter_lst[[2]] <- filter
names(filter_lst)[2] <- "not lost to followup"

filter <- filter & sample_qc$putative.sex.chromosome.aneuploidy == 0
filter_lst[[3]] <- filter
names(filter_lst)[3] <- "no aneuploidy in sex chromosome"

filter <- filter & sample_qc$Submitted.Gender == sample_qc$Inferred.Gender
filter_lst[[4]] <- filter
names(filter_lst)[4] <- "reported sex matches genetic sex"

filter <- filter & sample_qc$het.missing.outliers == 0
filter_lst[[5]] <- filter
names(filter_lst)[5] <- "no het missing outliers"

filter <- filter & sample_qc$excluded.from.kinship.inference == 0
filter_lst[[6]] <- filter
names(filter_lst)[6] <- "not excluded from kinship inference"

filter <- filter & sample_qc$excess.relatives == 0
filter_lst[[7]] <- filter
names(filter_lst)[7] <- "no excess relatives"

filter <- filter & sample_qc$in.Phasing.Input.chr1_22 == 1 & 
    sample_qc$in.Phasing.Input.chrX == 1 &
    sample_qc$in.Phasing.Input.chrXY == 1
filter_lst[[8]] <- filter
names(filter_lst)[8] <- "in phasing input"

filter <- filter & all_eids %in% panukb_matched$eid[which(panukb_matched$pop == "EUR")]
filter_lst[[9]] <- filter
names(filter_lst)[9] <- "in panukb EUR"

filter <- filter & bd[, f_assesscentre] %in% assess_centre_England
filter_lst[[10]] <- filter
names(filter_lst)[10] <- "in assessment centre England"



In [37]:
sgss_filter <- sgss
hes_filter <- hes_infect
ukb_filter <- list()
for (i in 1:length(filter_lst)) {
    filter <- filter_lst[[i]]
    eids_filtered <- unique(all_eids[filter])
    sgss_filter <- sgss_filter[sgss_filter$UKB_EID %in% eids_filtered, ]
    hes_filter <- hes_filter[hes_filter$eid %in% eids_filtered, ]
    sgss_cnt <- length(unique(sgss_filter$UKB_EID))
    hes_cnt <- length(unique(hes_filter$eid))
    ukb_filter[[i]] <- c(length(eids_filtered), sgss_cnt, hes_cnt)
}
ukb_filter <- do.call(rbind, ukb_filter)
rownames(ukb_filter) <- names(filter_lst)

ukb_filter

all,502505,114736,69900
not lost to followup,426755,108226,56142
no aneuploidy in sex chromosome,414121,104782,54100
reported sex matches genetic sex,413957,104733,54067
no het missing outliers,413123,104500,53948
not excluded from kinship inference,413115,104498,53948
no excess relatives,412956,104456,53924
in phasing input,412721,104385,53878
in panukb EUR,359929,92639,46432
in assessment centre England,359929,92639,46432


### Pathogen filtering

In [38]:
# for SGSS
sgss_tax_file <- paste0(bugbank_data_dir, "bb_pathogen_taxonomy_13032023.tsv")
sgss_tax <- read.table(sgss_tax_file, sep = "\t", header = TRUE, stringsAsFactors = FALSE)

# ------------------------------------------------------------
# create a dictionary from origin_name to most specific taxa
# ------------------------------------------------------------

# sanity check: no NA nor duplicated origin_name
sum(is.na(sgss_tax$origin_name))
sum(duplicated(sgss_tax$origin_name))
origin_name_to_tax <- list()

# for each origin_name, find the most specific tax

# get the tax levels
tax_levs <- colnames(sgss_tax)[-ncol(sgss_tax)]
# remove the last column (which is the origin_name) for later use
sgss_tax_only <- sgss_tax[, -ncol(sgss_tax)]

for (i in 1:nrow(sgss_tax)) {
    origin_name <- sgss_tax$origin_name[i]
    # remove any NA and the last column (which is the origin_name)
    tax <- sgss_tax_only[i, ]
    tax_lev <- tax_levs[!is.na(tax)]
    tax <- tax[!is.na(tax)]
    specific_tax <- tail(tax, 1)
    specific_tax_lev <- tail(tax_lev, 1)
    origin_name_to_tax[[origin_name]] <- c(specific_tax, specific_tax_lev)
}



[1] 0

[1] 0

In [57]:
# filter1: starting from all
filter_lst <- list()
filter <- rep(T, nrow(sgss))

filter_lst[[1]] <- filter
names(filter_lst)[1] <- "all"

# filter2: only include individuals that passed the QC
filter_lst[[2]] <- filter & sgss$UKB_EID %in% eids_filtered
names(filter_lst)[2] <- "individual that passed QC"

# filter3: only include infection that have a pathogen label in species level
sgss$tax_lev <- map_chr(sgss$ORGANISM_SPECIES_NAME, function(origin_name) {
    tax_lev <- origin_name_to_tax[[origin_name]][2]
})
filter_lst[[3]] <- filter_lst[[2]] & sgss$tax_lev == "species"
names(filter_lst)[3] <- "species level pathogen"

# filter4: only include pathogen that have infection cases > 100
sgss$tax <- map_chr(sgss$ORGANISM_SPECIES_NAME, function(origin_name) {
    species_name <- origin_name_to_tax[[origin_name]][1]
})
species_name_qc <- sgss$tax[filter_lst[[3]]]
species_name_freq <- table(species_name_qc)
species_name_gwas <- names(species_name_freq)[which(species_name_freq > 100)]
filter_lst[[4]] <- sgss$tax %in% species_name_gwas
names(filter_lst)[4] <- "pathogen with > 100 cases"

sgss_filter_table <- list()
for(i in 1:length(filter_lst)) {
    filter <- filter_lst[[i]]
    record_cnt <- sum(filter, na.rm = TRUE)
    ind_cnt <- length(unique(sgss$UKB_EID[filter]))
    tax_cnt <- length(unique(sgss$tax[filter]))
    sgss_filter_table[[i]] <- c(record_cnt, ind_cnt, tax_cnt)
}
sgss_filter_table <- do.call(rbind, sgss_filter_table)
rownames(sgss_filter_table) <- names(filter_lst)
colnames(sgss_filter_table) <- c("record_cnt", "ind_cnt", "tax_cnt")
sgss_filter_table





,record_cnt,ind_cnt,tax_cnt
all,350699,114737,526
individual that passed QC,281762,92639,488
species level pathogen,217109,78310,388
pathogen with > 100 cases,263908,95469,34


[1] 36

In [ ]:
# sanity check, see if the resulting number of tax is the same as the number of GWAS
gwas_wrkdir <- "/well/bag/clme1992/saige_pipe_test"
gwas_files <- list.files(gwas_wrkdir, pattern = "summary.05062023_sgss_species.*all.txt.gz", full.names = TRUE)
gwas_files_species <- gsub(gwas_files, "summary.05062023_sgss_species", "")
gwas_files_species <- gsub(gwas_files, ".species.all.txt.gz", "")
gwas_files_species <- gsub(gwas_files, ".species.all.txt.gz", "")
gwas_files_species

[1] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Acinetobacter_baumannii.species.all.txt.gz"     
 [2] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Bacteroides_fragilis.species.all.txt.gz"        
 [3] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Campylobacter_jejuni.species.all.txt.gz"        
 [4] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Candida_albicans.species.all.txt.gz"            
 [5] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Candida_parapsilosis.species.all.txt.gz"        
 [6] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Citrobacter_freundii.species.all.txt.gz"        
 [7] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Citrobacter_koseri.species.all.txt.gz"          
 [8] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Corynebacterium_striatum.species.all.txt.gz"    
 [9] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Cutibacterium_acnes.species.all.txt.gz"         
[10] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Enterobacter_cloacae.species.all.txt.gz"        
[11] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Enterococcus_faecalis.species.all.txt.gz"       
[12] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Enterococcus_faecium.species.all.txt.gz"        
[13] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Escherichia_coli.species.all.txt.gz"            
[14] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Haemophilus_influenzae.species.all.txt.gz"      
[15] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Haemophilus_parainfluenzae.species.all.txt.gz"  
[16] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Klebsiella_aerogenes.species.all.txt.gz"        
[17] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Klebsiella_oxytoca.species.all.txt.gz"          
[18] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Klebsiella_pneumoniae.species.all.txt.gz"       
[19] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Klebsiella_variicola.species.all.txt.gz"        
[20] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Micrococcus_luteus.species.all.txt.gz"          
[21] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Moraxella_catarrhalis.species.all.txt.gz"       
[22] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Morganella_morganii.species.all.txt.gz"         
[23] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Nakaseomyces_glabratus.species.all.txt.gz"      
[24] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Proteus_mirabilis.species.all.txt.gz"           
[25] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Proteus_vulgaris.species.all.txt.gz"            
[26] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Pseudomonas_aeruginosa.species.all.txt.gz"      
[27] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Pseudomonas_putida.species.all.txt.gz"          
[28] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Raoultella_ornithinolytica.species.all.txt.gz"  
[29] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Salmonella_enterica.species.all.txt.gz"         
[30] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Serratia_liquefaciens.species.all.txt.gz"       
[31] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Serratia_marcescens.species.all.txt.gz"         
[32] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Staphylococcus_aureus.species.all.txt.gz"       
[33] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Stenotrophomonas_maltophilia.species.all.txt.gz"
[34] "/well/bag/clme1992/saige_pipe_test/

In [56]:
species_name_freq[which(names(species_name_freq) == "Pasteurella multocida")]
species_name_freq[which(names(species_name_freq) == "Escherichia coli")]

Pasteurella multocida 
                  122

Escherichia coli 
          141296

In [45]:
pheno_rds_file <- paste0(gwas_wrkdir, "/log.ukb41482.bd.gwasdata.05062023_sgss_species.rds")
pheno_rds <- readRDS(pheno_rds_file)

names(pheno_rds$cases.count)

[1] "Escherichia_coli.species.URINE"                        
  [2] "Escherichia_coli.species.BLOOD"                        
  [3] "Escherichia_coli.species.MSU"                          
  [4] "Escherichia_coli.species.SPUTUM"                       
  [5] "Escherichia_coli.species.SWAB___NOS"                   
  [6] "Escherichia_coli.species.RECTUM"                       
  [7] "Escherichia_coli.species.CSU"                          
  [8] "Escherichia_coli.species.FLUID___NOS"                  
  [9] "Escherichia_coli.species.PUS_SOURCE_UNKNOWN"           
 [10] "Escherichia_coli.species.WOUND_SURGICAL"               
 [11] "Escherichia_coli.species.WOUND_TRAUMATIC"              
 [12] "Escherichia_coli.species.TISSUE"                       
 [13] "Escherichia_coli.species.UNKNOWN"                      
 [14] "Escherichia_coli.species.URINARY_CATHETER"             
 [15] "Escherichia_coli.species.SKIN_WOUND"                   
 [16] "Escherichia_coli.species.all"                          
 [17] "Enterobacter_cloacae.species.URINE"                    
 [18] "Enterobacter_cloacae.species.WOUND_SURGICAL"           
 [19] "Enterobacter_cloacae.species.SPUTUM"                   
 [20] "Enterobacter_cloacae.species.WOUND_TRAUMATIC"          
 [21] "Enterobacter_cloacae.species.BLOOD"                    
 [22] "Enterobacter_cloacae.species.MSU"                      
 [23] "Enterobacter_cloacae.species.all"                      
 [24] "Klebsiella_pneumoniae.species.URINE"                   
 [25] "Klebsiella_pneumoniae.species.SPUTUM"                  
 [26] "Klebsiella_pneumoniae.species.BLOOD"                   
 [27] "Klebsiella_pneumoniae.species.SWAB___NOS"              
 [28] "Klebsiella_pneumoniae.species.WOUND_TRAUMATIC"         
 [29] "Klebsiella_pneumoniae.species.WOUND_SURGICAL"          
 [30] "Klebsiella_pneumoniae.species.MSU"                     
 [31] "Klebsiella_pneumoniae.species.all"                     
 [32] "Klebsiella_oxytoca.species.URINE"                      
 [33] "Klebsiella_oxytoca.species.BLOOD"                      
 [34] "Klebsiella_oxytoca.species.MSU"                        
 [35] "Klebsiella_oxytoca.species.SPUTUM"                     
 [36] "Klebsiella_oxytoca.species.all"                        
 [37] "Klebsiella_aerogenes.species.URINE"                    
 [38] "Klebsiella_aerogenes.species.SPUTUM"                   
 [39] "Klebsiella_aerogenes.species.MSU"                      
 [40] "Klebsiella_aerogenes.species.all"                      
 [41] "Klebsiella_variicola.species.URINE"                    
 [42] "Klebsiella_variicola.species.all"                      
 [43] "Citrobacter_koseri.species.URINE"                      
 [44] "Citrobacter_koseri.species.SPUTUM"                     
 [45] "Citrobacter_koseri.species.MSU"                        
 [46] "Citrobacter_koseri.species.all"                        
 [47] "Citrobacter_freundii.species.URINE"                    
 [48] "Citrobacter_freundii.species.MSU"                      
 [49] "Citrobacter_freundii.species.all"                      
 [50] "Salmonella_enterica.species.all"                       
 [51] "Raoultella_ornithinolytica.species.all"                
 [52] "Proteus_mirabilis.species.URINE"                       
 [53] "Proteus_mirabilis.species.BLOOD"                       
 [54] "Proteus_mirabilis.species.SPUTUM"                      
 [55] "Proteus_mirabilis.species.WOUND_SURGICAL"              
 [56] "Proteus_mirabilis.species.MSU"                         
 [57] "Proteus_mirabilis.species.all"                         
 [58] "Proteus_vulgaris.species.all"                          
 [59] "Morganella_morganii.species.URINE"                     
 [60] "Morganella_morganii.species.MSU"                       
 [61] "Morganella_morganii.species.all"                       
 [62] "Serratia_marcescens.species.SPUTUM"                    
 [63] "Serratia_marcescens.species.URINE"                     
 [64] "Serratia_marcescens.specie

In [53]:
test <- sgss[which(sgss$tax == "Escherichia coli"), ]
dim(test)

[1] 141296     15

In [49]:
test <- sgss[which(sgss$tax == "Pasteurella multocida"), ]
dim(test)

[1] 122  15

In [35]:
unique(sgss$tax[filter])

[1] "Escherichia coli"             "Staphylococcus aureus"       
 [3] "Citrobacter koseri"           "Serratia marcescens"         
 [5] "Citrobacter freundii"         "Klebsiella pneumoniae"       
 [7] "Morganella morganii"          "Pseudomonas aeruginosa"      
 [9] "Haemophilus influenzae"       "Proteus mirabilis"           
[11] "Enterococcus faecalis"        "Enterobacter cloacae"        
[13] "Bacteroides fragilis"         "Streptococcus intermedius"   
[15] "Cutibacterium acnes"          "Klebsiella oxytoca"          
[17] "Corynebacterium striatum"     "Acinetobacter baumannii"     
[19] "Stenotrophomonas maltophilia" "Candida albicans"            
[21] "Moraxella catarrhalis"        "Aspergillus fumigatus"       
[23] "Enterococcus faecium"         "Nakaseomyces glabratus"      
[25] "Klebsiella variicola"         "Pseudomonas putida"          
[27] "Streptococcus pneumoniae"     "Streptococcus milleri"       
[29] "Klebsiella aerogenes"         "Campylobacter jejuni"        
[31] "Haemophilus parainfluenzae"   "Salmonella enterica"         
[33] "Proteus vulgaris"             "Candida parapsilosis"        
[35] "Micrococcus luteus"           "Raoultella ornithinolytica"  
[37] "Pasteurella multocida"        "Serratia liquefaciens"

In [33]:
gwas_files

[1] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Acinetobacter_baumannii.species.all.txt.gz"     
 [2] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Bacteroides_fragilis.species.all.txt.gz"        
 [3] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Campylobacter_jejuni.species.all.txt.gz"        
 [4] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Candida_albicans.species.all.txt.gz"            
 [5] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Candida_parapsilosis.species.all.txt.gz"        
 [6] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Citrobacter_freundii.species.all.txt.gz"        
 [7] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Citrobacter_koseri.species.all.txt.gz"          
 [8] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Corynebacterium_striatum.species.all.txt.gz"    
 [9] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Cutibacterium_acnes.species.all.txt.gz"         
[10] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Enterobacter_cloacae.species.all.txt.gz"        
[11] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Enterococcus_faecalis.species.all.txt.gz"       
[12] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Enterococcus_faecium.species.all.txt.gz"        
[13] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Escherichia_coli.species.all.txt.gz"            
[14] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Haemophilus_influenzae.species.all.txt.gz"      
[15] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Haemophilus_parainfluenzae.species.all.txt.gz"  
[16] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Klebsiella_aerogenes.species.all.txt.gz"        
[17] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Klebsiella_oxytoca.species.all.txt.gz"          
[18] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Klebsiella_pneumoniae.species.all.txt.gz"       
[19] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Klebsiella_variicola.species.all.txt.gz"        
[20] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Micrococcus_luteus.species.all.txt.gz"          
[21] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Moraxella_catarrhalis.species.all.txt.gz"       
[22] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Morganella_morganii.species.all.txt.gz"         
[23] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Nakaseomyces_glabratus.species.all.txt.gz"      
[24] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Proteus_mirabilis.species.all.txt.gz"           
[25] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Proteus_vulgaris.species.all.txt.gz"            
[26] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Pseudomonas_aeruginosa.species.all.txt.gz"      
[27] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Pseudomonas_putida.species.all.txt.gz"          
[28] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Raoultella_ornithinolytica.species.all.txt.gz"  
[29] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Salmonella_enterica.species.all.txt.gz"         
[30] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Serratia_liquefaciens.species.all.txt.gz"       
[31] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Serratia_marcescens.species.all.txt.gz"         
[32] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Staphylococcus_aureus.species.all.txt.gz"       
[33] "/well/bag/clme1992/saige_pipe_test/summary.05062023_sgss_species.Stenotrophomonas_maltophilia.species.all.txt.gz"
[34] "/well/bag/clme1992/saige_pipe_test/

In [19]:
# load HES
start_time = Sys.time()
hes_file = paste0(ukb_data_dir, "hes/hes_diag.latest.txt.gz")
hes = read.table(hes_file, sep = "\t", header = TRUE, stringsAsFactors = FALSE)
end_time = Sys.time()
print(paste0("Time taken to load hes: ", end_time - start_time))

[1] "Time taken to load HESIN: 17.3452785015106"


In [12]:
# load icd10 data
icd10_desc_file <- paste0(bugbank_data_dir, "pathogen_to_unique_icd10.tsv")
icd10_desc <- read.table(icd10_desc_file, sep = "\t", header = TRUE, stringsAsFactors = FALSE)

In [33]:
# subset hes to only infection related
infect_icd10_codes <- unique(unlist(strsplit(icd10_desc$icd10, split = ",")))
hes_infect <- hes[hes$diag_icd10 %in% infect_icd10_codes, ]

# number of records
print(paste0("Number of infection related records: ", nrow(hes_infect)))

# number of unique individuals
print(paste0("Number of unique individuals: ", length(unique(hes_infect$eid))))

# number of unique icd10 codes
print(paste0("Number of unique icd10 codes: ", length(unique(hes_infect$diag_icd10))))

# number of species level pathogens
# create a dictionary of ICD-10 to pathogen mapping
icd_to_pathogen = list()
for (i in 1:nrow(icd10_desc)) {
  cur_icd10s = unlist(strsplit(icd10_desc$icd10[i], ","))
  for (icd10 in cur_icd10s) {
    icd_to_pathogen[[icd10]] = c(icd10_desc$org_name[i], icd10_desc$tax_lev[i])
  }
}
# map hes icd10 codes to pathogen
hes_infect$org_name = map_chr(hes_infect$diag_icd10, function(x) icd_to_pathogen[[x]][1])
print(paste0("Number of unique pathogens: ", length(unique(hes_infect$org_name))))

# number of species level pathogen
hes_infect$tax_lev = map_chr(hes_infect$diag_icd10, function(x) icd_to_pathogen[[x]][2])
hes_infect_species = hes_infect[hes_infect$tax_lev == "species", ]
print(x = paste0("Number of unique species level pathogens: ", length(unique(hes_infect_species$org_name))))


[1] "Number of infection related records: 188788"
[1] "Number of unique individuals: 69900"
[1] "Number of unique icd10 codes: 480"
[1] "Number of unique pathogens: 155"
[1] "Number of unique species level pathogens: 88"


In [32]:
head(hes_infect_species)

,eid,ins_index,arr_index,level,diag_icd9,diag_icd9_nb,diag_icd10,diag_icd10_nb,org_name,tax_lev
,<int>,<int>,<int>,<int>,<chr>,<lgl>,<chr>,<int>,<chr>,<chr>
2750,1000906,6,5,2,,NA,B962,NA,Escherichia coli,species
4438,1001488,10,1,2,,NA,B961,NA,Klebsiella pneumoniae,species
4455,1001488,11,1,2,,NA,B961,NA,Klebsiella pneumoniae,species
4702,1001603,2,2,2,,NA,B962,NA,Escherichia coli,species
5375,1001970,8,4,2,,NA,B962,NA,Escherichia coli,species
5712,1002128,7,3,2,,NA,B171,NA,Hepacivirus C,species
